In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.abspath('../src/'))
sys.path.append(os.path.abspath('../'))

In [2]:
from src.central_ids import IDS
from src.utils import print_results
from src.dataset import CANDataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import numpy as np
from pathlib import Path
from sklearn.metrics import f1_score, confusion_matrix
import pickle
import torch
from flwr.common.parameter import parameters_to_ndarrays
from collections import OrderedDict
from utils import set_parameters, test_model

from pytorch_lightning import loggers
from pytorch_lightning.callbacks import ModelCheckpoint

In [3]:
car_model = 'Tesla'
rnd = 49
args = {
    'ckpt_file' : f'../save/FedAvg/train40000_50rnds_15epochs_lr0.0005/ckpt-rnd{rnd}.npz',
    'train_dir': f'../../Data/LISA/{car_model}/train_40000/',
    'val_dir': f'../../Data/LISA/{car_model}/val_10000/',
    'save_dir': f'../save/FedAvg_FN/{car_model}/',
    'C': 2,
    'batch_size': 64,
    'epochs': 10,
    'val_freq': 1,
    'lr': 0.005,
    'weight_decay': 1e-3
}

# Fine tuning the federated model

In [4]:
# ckpt_file = f'../save/FedAvg/train1000_100rnds/ckpt-rnd{rnd}.npz'
tensors = np.load(args['ckpt_file'], allow_pickle=True)['arr_0'][()]
model = IDS(**args)

In [5]:
weights = parameters_to_ndarrays(tensors)
params_dict = zip(model.state_dict().keys(), weights)
state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
model.load_state_dict(state_dict, strict=True)

<All keys matched successfully>

In [6]:
for name, param in model.named_parameters():
    if 'fc' in name:
        param.requires_grad = False
    else:
        param.requires_grad = True

In [7]:
logger = loggers.TensorBoardLogger(save_dir=args['save_dir'],) 
checkpoint_callback = ModelCheckpoint(
        filename='{epoch:02d}-{val_loss:.2f}-{val_f1:.4f}',
        every_n_epochs=args['val_freq'],
        monitor='val_loss', 
        save_top_k=5)

In [8]:
trainer = pl.Trainer(max_epochs=args['epochs'], accelerator='cuda', 
                        gpus=[0], logger=logger, log_every_n_steps=100, 
                        check_val_every_n_epoch=args['val_freq'],
                        callbacks=[checkpoint_callback])

/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/core/optimizer.py:331: RuntimeWarning: The lr scheduler dict contains the key(s) ['monitor'], but the keys will be ignored. You need to call `lr_scheduler.step()` manually in manual optimization.
  category=RuntimeWarning,

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Classifier       | 700 K 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
699 K     Trainable params
258       Non-trainable params
700 K     Total params
2.801     Total estimated model params size (MB)


Set the optimzer with the lr =  0.005


Sanity Checking: 0it [00:00, ?it/s]

/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Validation f1 score: 0.0000


...

Validation: 0it [00:00, ?it/s]

Validation f1 score: 0.9717


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


Validation f1 score: 0.9777


In [10]:
data_dir = f'../../Data/LISA/{car_model}/test_10000'
test_model(data_dir, model)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(array([[7164,   27],
        [  30, 2353]]),
 (0.988655462184874, 0.9874108266890474, 0.9880327524669326))

# Run model from checkpoint

In [6]:
ckpt_files = '../save/FedAvg_FN/Kia/lightning_logs/version_0/checkpoints/epoch=09-val_loss=0.00-val_f1=0.9980.ckpt'
fedavg_model = IDS.load_from_checkpoint(ckpt_files, **args)
data_dir = f'../../Data/LISA/Kia/'
print('Training set: ')
print(test_model(Path(data_dir) / 'train_50000', fedavg_model))
print('Test set: ')
print(test_model(Path(data_dir) / 'test_10000', fedavg_model))

Training set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(array([[37160,     0],
       [    0,  7317]]), (1.0, 1.0, 1.0))
Test set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(array([[7189,    2],
       [  21, 2362]]), (0.9991539763113367, 0.9911875786823332, 0.9951548346323994))


In [7]:
ckpt_files = '../save/FedAvg_FN/Tesla/lightning_logs/version_0/checkpoints/epoch=09-val_loss=0.02-val_f1=0.9921.ckpt'
fedavg_model = IDS.load_from_checkpoint(ckpt_files, **args)
data_dir = f'../../Data/LISA/Tesla/'
print('Training set: ')
print(test_model(Path(data_dir) / 'train_50000', fedavg_model))
print('Test set: ')
print(test_model(Path(data_dir) / 'test_10000', fedavg_model))

Training set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(array([[38458,    12],
       [    0,  9964]]), (0.9987971130713713, 1.0, 0.9993981945837512))
Test set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(array([[7747,   19],
       [  16, 2216]]), (0.9914988814317673, 0.992831541218638, 0.9921647638235954))


In [ ]:
ckpt_files = '../save/FedAvg_FN/BMW/lightning_logs/version_0/checkpoints/epoch=09-val_loss=0.03-val_f1=0.9852.ckpt'
fedavg_model = IDS.load_from_checkpoint(ckpt_files, **args)
data_dir = f'../../Data/LISA/BMW/'
print('Training set: ')
print(test_model(Path(data_dir) / 'train_50000', fedavg_model))
print('Test set: ')
print(test_model(Path(data_dir) / 'test_10000', fedavg_model))

Training set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(array([[30230,     7],
       [   33, 14181]]), (0.9995066253171694, 0.9976783452933727, 0.9985916484754596))
Test set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(array([[7664,   21],
       [  68, 2246]]), (0.9907366563740626, 0.9706136560069144, 0.9805719275267408))
